In [3]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [4]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(112, 112, 3)),
])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 32)      896       
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('./MODELS/conv2d.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmpkpgtt6tu/assets


그래서 해야할것을 조금이라도 정리해보자면..

1. 모델을 여러개 열심히 만든다 (up to 1000개? 너무 많은거 아니념)
2. 

In [34]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

kernel_size = [2*i + 1 for i in range(10)]
filter_size = [2**i for i in range(4, 8)]
input_img_size += [(i, i, 3) for i in range(32, 512)]

In [35]:
from itertools import product

comb = list(product(kernel_size, filter_size, input_img_size))

In [36]:
print(len(comb))
comb[:5]

116960


[(1, 16, (16, 16, 3)),
 (1, 16, (32, 32, 3)),
 (1, 16, (64, 64, 3)),
 (1, 16, (128, 128, 3)),
 (1, 16, (256, 256, 3))]

In [31]:
comb = comb[:5]

In [38]:
data_frame = {
    'tflite_name' : [],
    'kernel' : [],
    'filter' : [],
    'input_shape' : [],
    'latency' : [], # target value
}

In [40]:
import subprocess
def run_bazel(model_path):
    proc = subprocess.Popen(
        ['gsoc_proj/run.sh ./gsoc_proj/{}'.format(model_path)],
        stdout = subprocess.PIPE
    )
    out, err = proc.communicate()
    print(out.decode('utf-8'))

In [ ]:
run_bazel(./gsoc_proj)

In [33]:
import subprocess
for com in comb:
    _kernel, _filter, _input_shape = com
    model = keras.Sequential([
        keras.layers.Conv2D(filters=_filter, kernel_size=_kernel, padding='same', activation='relu', input_shape=_input_shape),
    ])
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    
    model_name = "kernel_{}_filter_{}_input_shape_{}.tflite".format(_kernel, _filter, _input_shape)
    with open("./MODELS/" + model_name, 'wb') as f:
        f.write(tflite_model)
    # json? 만들기.. 
    data_frame['tflite_name'].append(model_name)
    data_frame['kernel'].append(_kernel)
    data_frame['filter'].append(_fileter)
    data_frame['input_shape'].append(_input_shape)
    
    

INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmp993dina7/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmp993dina7/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmpl7uw55ey/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmpl7uw55ey/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmpgdzpmzmd/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmpgdzpmzmd/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmp4es70dmp/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmp4es70dmp/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmplb2x8hqf/assets


INFO:tensorflow:Assets written to: /var/folders/m0/rdxcs9f919x3p29r3c4mvz7w0000gn/T/tmplb2x8hqf/assets


In [41]:
proc = subprocess.Popen(
    ['cd ..'],
    ['gsoc_proj/run.sh ./gsoc_proj/conv_2d_tflite'],
    stdout = subprocess.PIPE
)
out, err = proc.communicate()
print(out.decode('utf-8'))

TypeError: bufsize must be an integer

In [ ]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

In [42]:
from tensorflow.lite.python import schema_py_generated as schema_fb

In [46]:
tflite_input = './conv_2d.tflite'

In [48]:
with open(tflite_input, 'rb') as file_handle:
    file_data = bytearray(file_handle.read())

In [51]:
def CamelCaseToSnakeCase(camel_case_input):
  """Converts an identifier in CamelCase to snake_case."""
  s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", camel_case_input)
  return re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1).lower()


def FlatbufferToDict(fb, preserve_as_numpy):
  """Converts a hierarchy of FB objects into a nested dict.
  We avoid transforming big parts of the flat buffer into python arrays. This
  speeds conversion from ten minutes to a few seconds on big graphs.
  Args:
    fb: a flat buffer structure. (i.e. ModelT)
    preserve_as_numpy: true if all downstream np.arrays should be preserved.
      false if all downstream np.array should become python arrays
  Returns:
    A dictionary representing the flatbuffer rather than a flatbuffer object.
  """
  if isinstance(fb, int) or isinstance(fb, float) or isinstance(fb, str):
    return fb
  elif hasattr(fb, "__dict__"):
    result = {}
    for attribute_name in dir(fb):
      attribute = fb.__getattribute__(attribute_name)
      if not callable(attribute) and attribute_name[0] != "_":
        snake_name = CamelCaseToSnakeCase(attribute_name)
        preserve = True if attribute_name == "buffers" else preserve_as_numpy
        result[snake_name] = FlatbufferToDict(attribute, preserve)
    return result
  elif isinstance(fb, np.ndarray):
    return fb if preserve_as_numpy else fb.tolist()
  elif hasattr(fb, "__len__"):
    return [FlatbufferToDict(entry, preserve_as_numpy) for entry in fb]
  else:
    return fb


def CreateDictFromFlatbuffer(buffer_data):
  model_obj = schema_fb.Model.GetRootAsModel(buffer_data, 0)
  model = schema_fb.ModelT.InitFromObj(model_obj)
  return FlatbufferToDict(model, preserve_as_numpy=False)

In [53]:
import re
data = CreateDictFromFlatbuffer(file_data)

In [57]:
data.keys()

dict_keys(['buffers', 'description', 'metadata', 'metadata_buffer', 'operator_codes', 'signature_defs', 'subgraphs', 'version'])

In [66]:
for dic in data['subgraphs']:
    print(dic.keys())

dict_keys(['inputs', 'name', 'operators', 'outputs', 'tensors'])


In [59]:
data['metadata_buffer']

In [68]:
len(data['subgraphs'])

1

In [69]:
data

{'buffers': [{'data': None},
  {'data': array([154, 224, 234, 190, 186, 156, 181,  62, 165,  62,  10,  64,  67,
           95, 193, 191, 132,   0, 133, 190,  96,  33,  18,  63, 241, 171,
           57,  64, 119,  46,  77,  64, 170, 189, 109, 191, 212, 110,  50,
           64,  59, 158,  35,  64,  81, 135, 219,  61,  76, 231,  72,  64,
           81, 186,  66,  64, 238, 177,  23,  64,  84, 240, 179,  60, 197,
           50,  56, 192, 168, 143,  29,  64,  29, 151,  81,  64,  70,  95,
           86,  64,  18, 153,  31, 191, 162,  34, 202, 191,   1,  96, 233,
           63, 124,  27,  16,  64,  91,   9, 127,  63, 220, 173,  67,  64,
           32, 192,  17,  64, 201,  95,   8,  64,  67, 240,  89,  64,  80,
          206,  30,  64, 187, 245, 232,  63, 140, 255, 134,  63], dtype=uint8)},
  {'data': None},
  {'data': array([ 72, 153,  63, ..., 232,  41,  59], dtype=uint8)},
  {'data': None}],
 'description': [84,
  79,
  67,
  79,
  32,
  67,
  111,
  110,
  118,
  101,
  114,
  116,
  101,
 

In [78]:
kernel_list = [1, 3, 5]
filter_list = [16, 32, 64]
input_hw = [8, 16, 32, 64]
input_channels = [16, 32, 48]
input_img_list = []
for hw in input_hw:
    for ch in input_channels:
        input_img_list.append((ch, hw, hw))
stride = [1, 2, 3]
padding = ['same', 'valid']

In [77]:
from itertools import product
len(list(product(kernel_list, filter_list, input_img_list, stride, padding)))

648